In [ ]:
#install necessary packages
# !pip install selenium
# !apt-get update
# !apt-get install -y chromium-browser
# !pip install numpy
# !apt install chromium-chromedriver
# !pip install bs4
# !pip install ichromedriver-autoinstaller

: 

In [1]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install() 

update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/local/share/doc /usr/local/share/doc none bind,ro 0 0): cannot open directory "/usr/local/share": permission denied
update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/share/gimp/2.0/help /usr/share/gimp/2.0/help none bind,ro 0 0): cannot open directory "/var/lib/snapd": permission denied
update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/share/xubuntu-docs /usr/share/xubuntu-docs none bind,ro 0 0): cannot open directory "/var/lib/snapd": permission denied


'/home/felix/.local/lib/python3.11/site-packages/chromedriver_autoinstaller/123/chromedriver'

In [2]:
#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import requests
import re
from bs4 import BeautifulSoup
import json
import os
from multiprocessing import Pool
import numpy as np

In [16]:
#initialize constants

request_credentials = {
    "username": 'profesor',
    "password": 'profe2020',
    "profile": 'docente'
}

base_script = """
const dataP = {};
var dummy = "";

function getMaxValue(dataPoints) {
    if (Array.isArray(dataPoints[0].data)) {
        return Math.max(...dataPoints.map(dataset => Math.max(...dataset.data)));
    } else {
        return Math.max(...dataPoints);
    }
}

function designGraphics(idName, format, type, coordsX, dataPoints, stepSize, max, sigDif = null, dif = null, mDif =
        null) {
        const data = {
          format: format,
          coordsX: coordsX,
          dataPoints: dataPoints,
          sigDif: sigDif,
          dif: dif,
          mDif: mDif
        }

        dataP[idName] = data;
    }
"""

end_script = """
scrapFunction();

return dataP
"""

instructions_to_replace = [
    "document.getElementById('div-' + i + '-2').style.marginTop",
    "document.getElementById('dif-' + i + '-2').innerHTML",
    "container.innerHTML",
    "padreCanvas.innerHTML",
    "document.Fdocument('dif-' + i + '-2').innerHTML",
    "canvasContainer.parentNode"
  ]
replacement = "dummy"

pattern = r"\b(?:" + "|".join(map(re.escape, instructions_to_replace)) + r")\b"

opt = webdriver.ChromeOptions()
opt.add_argument('--headless')
opt.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
opt.add_argument("--remote-debugging-port=9222")

n_workers = max(os.cpu_count() - 2, 1)

In [17]:
def authUrl(rbd):
    queryParams = 'auth?client_id=agencia&response_type=code&state=1'
    redirect = f'redirect_uri=http://www.simce.cl/validation/{rbd}'
    baseUrl = 'https://perfilador.agenciaeducacion.cl/auth/realms/Perfilador/protocol/openid-connect/'
    return f'{baseUrl}{queryParams}&{redirect}'

def set_script(script):
  index = script.find("document.addEventListener('DOMContentLoaded', function()")
  script = script[index:]\
    .replace("document.addEventListener('DOMContentLoaded', function()", "function scrapFunction()")\
    .replace("});\n\n    window.onload", "};\n\n    window.onload")
  end_index = script.find("window.onload")
  script = re.sub(pattern, replacement, script[:end_index])
  return base_script + "\n\n" + script + "\n\n" + end_script

def scrap(rbd):
  s = requests.Session()
  res = s.get(authUrl(rbd))
  log_url = BeautifulSoup(res.content).find('form')['action']
  s.post(log_url, data=request_credentials)
  landing = BeautifulSoup(s.get(f'https://www.simce.cl/{rbd}/inicio').content, 'html.parser')
  if landing.find_all('script')[-1].text.find("(parseInt('2023')") == -1:
    return -1

  indicator_page = BeautifulSoup(s.get(f'https://www.simce.cl/{rbd}/indicador').content, 'html.parser')
  dimension_page = BeautifulSoup(s.get(f'https://www.simce.cl/{rbd}/dimension').content, 'html.parser')

  school_type = indicator_page.find(id='dependencia').text
  municipality = indicator_page.find(id='comuna').text
  gse = indicator_page.find_all(class_='level-title mb-3')
  gse = list(filter(lambda x: 'GSE' in x.text, gse))[0].find(id='selectedOption').text

  indicator_script = indicator_page.find_all('script')[-1].get_text()
  dimension_script = dimension_page.find_all('script')[-1].get_text()
  driver = webdriver.Chrome(options=opt)
  indicators = driver.execute_script(set_script(indicator_script))
  dimensions = driver.execute_script(set_script(dimension_script))
  driver.stop_client()
  driver.close()
  driver.quit()
  return {
    'indicators': indicators,
    'dimensions': dimensions,
    'school_type': school_type,
    'municipality': municipality,
    'GSE': gse
  }

def process_schools(schools):
  errors = []
  no_data = []
  results = []
  for school in schools:
    try:
      scrap_data = scrap(str(school['rbd']))
      if scrap_data == -1:
        no_data.append(str(school['rbd']))
        continue

      data = {
          'nombre_colegio': school['rbd_nombre'],
          'rbd': school['rbd'],
          'dependencia': scrap_data['school_type'],
          'comuna': scrap_data['municipality'],
          'GSE': scrap_data['GSE'],
          'indicadores': scrap_data['indicators'],
          'dimensiones': scrap_data['dimensions'],
      }
      results.append(data)
    except Exception as e:
      print(f"error {e} on {school['rbd']}")
      errors.append([school['rbd'], e])
  return results, errors, no_data

def main():
  schools_file = open('establecimientos.json')
  schools = json.load(schools_file)
  schools_file.close()
  os.makedirs('jsons', exist_ok=True)
  errors = []
  no_data = []
  sub_schools_list = [schools[i*len(schools)//n_workers: (i+1)*len(schools)//n_workers] for i in range(n_workers)]
  with Pool(n_workers) as p:
    feedback = p.map(process_schools, sub_schools_list)
  results, errors, no_data = zip(*feedback)
  errors, no_data = [inner_list[0] for inner_list in errors if inner_list], [inner_list[0] for inner_list in no_data if inner_list]
  results = np.array(results).flatten()
  for i in results:
     with open(f"jsons/{i['rbd']}.json", "w", encoding='utf8') as output:
          json.dump(i, output, indent=2, ensure_ascii=False)
  if len(errors) > 0:
    with open(f"errores.txt", "w") as output:
          output.write('\n'.join(str(error) for error in errors))
  if len(no_data) > 0:
    with open(f"sin_datos.txt", "w") as output:
          output.write('\n'.join(no_data))

In [18]:
main()

error Message: disconnected: unable to connect to renderer
Stacktrace:
#0 0x5645044328a3 <unknown>
#1 0x5645041288c6 <unknown>
#2 0x5645041625a7 <unknown>
#3 0x56450415d6d3 <unknown>
#4 0x564504158d3d <unknown>
#5 0x5645041a1aed <unknown>
#6 0x564504195343 <unknown>
#7 0x564504166593 <unknown>
#8 0x564504166f5e <unknown>
#9 0x5645043f688b <unknown>
#10 0x5645043fa7e5 <unknown>
#11 0x5645043e45b1 <unknown>
#12 0x5645043fb372 <unknown>
#13 0x5645043c98bf <unknown>
#14 0x564504421768 <unknown>
#15 0x56450442193b <unknown>
#16 0x5645044319f4 <unknown>
#17 0x7f472248f6ba <unknown>
 on 7630
error Message: disconnected: unable to connect to renderer
Stacktrace:
#0 0x55af6e62c8a3 <unknown>
#1 0x55af6e3228c6 <unknown>
#2 0x55af6e35c5a7 <unknown>
#3 0x55af6e3576d3 <unknown>
#4 0x55af6e352d3d <unknown>
#5 0x55af6e39baed <unknown>
#6 0x55af6e38f343 <unknown>
#7 0x55af6e360593 <unknown>
#8 0x55af6e360f5e <unknown>
#9 0x55af6e5f088b <unknown>
#10 0x55af6e5f47e5 <unknown>
#11 0x55af6e5de5b1 <unknown>

KeyboardInterrupt: 

In [19]:
driver=webdriver.Chrome(options=opt)
driver.execute_script('return "hola"')
driver.quit()

WebDriverException: Message: disconnected: unable to connect to renderer
Stacktrace:
#0 0x55e67fb958a3 <unknown>
#1 0x55e67f88b8c6 <unknown>
#2 0x55e67f8c55a7 <unknown>
#3 0x55e67f8c06d3 <unknown>
#4 0x55e67f8bbd3d <unknown>
#5 0x55e67f904aed <unknown>
#6 0x55e67f8f8343 <unknown>
#7 0x55e67f8c9593 <unknown>
#8 0x55e67f8c9f5e <unknown>
#9 0x55e67fb5988b <unknown>
#10 0x55e67fb5d7e5 <unknown>
#11 0x55e67fb475b1 <unknown>
#12 0x55e67fb5e372 <unknown>
#13 0x55e67fb2c8bf <unknown>
#14 0x55e67fb84768 <unknown>
#15 0x55e67fb8493b <unknown>
#16 0x55e67fb949f4 <unknown>
#17 0x7f37c2a8f6ba <unknown>
